In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget


# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2)
    KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount)
Device succesfully connected (Rotation Mount)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(4)
TTC.set_alice_transmission_channel(3)
# set bob channels
TTC.set_bob_reflection_channel(2)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(6)
KMC.alice.home()


In [5]:
#TTC.displayCountTraces()
TTC.displayCoincidenceTraces()

Output()

In [6]:
idealAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> + |VV> initial state
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9)



corr[a,b] = 0.9277679100059206
	N[|T,T>]=1155
	N[|T,R>]=7
	N[|R,T>]=54
	N[|R,R>]=473

corr[a,B] = 0.5570307785153893
	N[|T,T>]=939
	N[|T,R>]=35
	N[|R,T>]=332
	N[|R,R>]=351

corr[A,b] = 0.3291316526610644
	N[|T,T>]=701
	N[|T,R>]=85
	N[|R,T>]=394
	N[|R,R>]=248

corr[A,B] = -0.515052888527258
	N[|T,T>]=207
	N[|T,R>]=124
	N[|R,T>]=807
	N[|R,R>]=91

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.3289832297096322


# OSC Server Setup


In [7]:
my_ip = '192.168.0.2'
my_port = 7401 

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=7400, name='Clemens')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [8]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Myself, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Myself


Received measurement: (1, 2, 3, 4)


In [10]:
while True:
    CHSH_angles = [0, -22.5, -11.5, 11.5]
    theta_a = np.random.choice(CHSH_angles[0:2])
    theta_b = np.random.choice(CHSH_angles[2:4])

    measurement_results = TTC.get_single_measurement(theta_a, theta_b)

    resultTranslate = {0 : [1, 1],
                1 : [1, -1],
                2 : [-1, 1],
                3 : [-1, -1]}

    resultClemensFormat = [1 if theta_a == CHSH_angles[0] else 2, 1 if theta_b == CHSH_angles[2] else 2 ,*resultTranslate[measurement_results]]

    OSC.send_measurement(Clemens, resultClemensFormat)


Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [2, 1, -1, 1] sent to Clemens
Measurement [1, 2, 1, 1] sent to Clemens
Measurement [2, 2, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 2, 1, 1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [2, 1, -1, 1] sent to Clemens
Measurement [2, 2, 1, 1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [1, 2, 1, 1] sent to Clemens
Measurement [2, 1, 1, -1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [2, 1, 1, -1] sent to Clemens
Measu

KeyboardInterrupt: 